In [1]:
require(tidyverse)
require(data.table)
require(forestplot)
require(meta)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: forestplot
Loading required package: grid
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: checkmate
Loading required package: meta
Loadi

In [2]:
madata <- data.frame(
    Variant = c('Gln175His', 'Arg177Ter', 'Arg140His', 'Gln136Arg'),
    TE      = c(-0.743050, -0.944409, -0.481583, -0.207756), 
    seTE    = c( 0.108010,  0.504579,  0.192698,  0.444267),
    stringsAsFactors=F
)


In [3]:
m<-metagen(
    TE,
    seTE,
    data=madata,
    studlab=paste(Variant),
    comb.fixed = TRUE,
    comb.random = FALSE,
    prediction=TRUE,
    sm="SMD"
)

In [4]:
m

              SMD             95%-CI %W(fixed)
Gln175His -0.7430 [-0.9547; -0.5314]      70.5
Arg177Ter -0.9444 [-1.9334;  0.0445]       3.2
Arg140His -0.4816 [-0.8593; -0.1039]      22.1
Gln136Arg -0.2078 [-1.0785;  0.6630]       4.2

Number of studies combined: k = 4

                        SMD             95%-CI     z  p-value
Fixed effect model  -0.6694 [-0.8471; -0.4917] -7.38 < 0.0001
Prediction interval         [-1.0595; -0.2793]               

Quantifying heterogeneity:
tau^2 = 0; H = 1.00 [1.00; 2.47]; I^2 = 0.0% [0.0%; 83.5%]

Test of heterogeneity:
    Q d.f. p-value
 2.79    3  0.4249

Details on meta-analytical method:
- Inverse variance method

In [5]:
m$pval.fixed

[1] 1.552256e-13

In [6]:
pooled_TE   = m$TE.fixed
pooled_seTE = m$seTE.fixed

In [7]:
tabletext<-cbind(
    c('Variant',    madata %>% pull(Variant), 'Summary'),
    c('IOP (mmHg)', madata %>% pull(TE) %>% lapply(round, digits =2) %>% lapply(as.character) %>% 
      simplify(), as.character(round(pooled_TE, digits=2)))
) 


In [8]:
plot_data <- bind_rows(
    data.frame(TE=NA, seTE=NA), 
    madata, 
    data.frame(TE=pooled_TE, seTE=pooled_seTE)
) %>%
rename(mean = TE) %>%
mutate(
    lower = mean - 1.96 * seTE,
    upper = mean + 1.96 * seTE
) %>%
select(mean, lower, upper)

In [9]:
png(file='ANGPTL7_IOP_forest.png', width=1600, height=1200, res=300) 
forestplot(
    tabletext, 
    plot_data,
    new_page = TRUE,
    is.summary=c(TRUE,rep(FALSE,madata %>% dim() %>% first()),TRUE),
    xlog=FALSE, 
    col=fpColors(box="royalblue",line="darkblue", summary="royalblue")
)
dev.off()

png 
  2

## reference 
- https://bookdown.org/MathiasHarrer/Doing_Meta_Analysis_in_R/fixed.html
- https://cran.r-project.org/web/packages/forestplot/vignettes/forestplot.html